# Zadanie dnia

# Zadanie 1

* Należy pobrać z wykorzystaniem pakietów requests, Beautiful Soup wszystkie wydarzenia zamieszczone w harmonogramie na stronie https://www.mimuw.edu.pl/. 
* Proszę napisać scraper w taki sposób, aby w efekcie działania zapisywał informacje jako lista słowników, których kluczami są tytuł, url do wydarzenia oraz jego data i godzina.
* Otrzymane dane proszę zapisać w formacie csv/xlsx - pomocne może być wykorzystanie ramki danych (pakiet pandas) i metody from_dict.
* Na koniec - prosze o wyświetlenie pięciu losowych wydarzeń z listy pobranych (tytuł, url, data).

# Zadanie 2

Należy wybrać dowolną stronę internetową (zgodnie z zainteresowaniami), która zezwala na pobieranie danych. Następnie proszę dokonać pobierania wybranych przez Państwa treści. Można spróbować wykorzystac Selenium, chociaż requests + Beautiful Soup są wystarczające. Uzasadnić do czego takie dane mogą się Państwu przydać.  

In [34]:
import requests
from bs4 import BeautifulSoup
def get_stock_data(ticker_symbol):
    url = f'https://finance.yahoo.com/quote/{ticker_symbol}'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    div = soup.find('div', {'class': 'container yf-1tejb6'})
    price = div.find_all('fin-streamer', {'class' : 'livePrice yf-1tejb6'})[0].get('data-value')
    price_change = div.find_all('fin-streamer', {'class' : 'priceChange yf-1tejb6'})[0].get('data-value')
    price_change_percent = div.find_all('fin-streamer', {'class' : 'priceChange yf-1tejb6'})[1].get('data-value')
    
    stock = {
    'name' : ticker_symbol,
    'price' : price,
    'change' : price_change,
    'change_percent' : price_change_percent
    }
    
    return stock

stock_symbols = ["AMZN", "TSLA", "BTC-USD", "META", "AAPL", "NFLX", "UEC"]
stock_data = []
for stock in stock_symbols:
    stock_data.append(get_stock_data(stock))

print(stock_data)

{'name': 'TSLA', 'price': '12.919983', 'change': '12.919983', 'change_percent': '3.8040226'}


In [43]:
from selenium import webdriver
from selenium.webdriver.chrome.service import  Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import TimeoutException

from webdriver_manager.chrome  import ChromeDriverManager # webdriver-manager is not required, but it makes managing web drivers way easier

ticker_symbol = 'AMZN'
url = f'https://finance.yahoo.com/quote/{ticker_symbol}'

# initialize a web driver instance
options = Options()
options.add_argument('--headless=new')
driver = webdriver.Chrome(
    service=ChromeService(ChromeDriverManager().install()),
    options=options
)
# set up the window size of the browser
driver.set_window_size(1920, 1080)
# visit the target page
driver.get(url)

try:
    # wait up to 3 seconds for the consent modal to show up
    consent_overlay = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.consent-overlay')))

    # click the "Accept all" button
    accept_all_button = consent_overlay.find_element(By.CSS_SELECTOR, '.accept-all')
    accept_all_button.click()
except TimeoutException:
    print('Cookie consent overlay missing')
    

regular_market_price = driver.find_element(
	By.CSS_SELECTOR,
	f'[data-symbol="{ticker_symbol}"][data-field="regularMarketPrice"]'
).text
regular_market_change = driver\
    .find_element(By.CSS_SELECTOR, f'[data-symbol="{ticker_symbol}"][data-field="regularMarketChange"]')\
    .text
regular_market_change_percent = driver\
    .find_element(By.CSS_SELECTOR, f'[data-symbol="{ticker_symbol}"][data-field="regularMarketChangePercent"]')\
    .text\
    .replace('(', '').replace(')', '')

post_market_price = driver\
    .find_element(By.CSS_SELECTOR, f'[data-symbol="{ticker_symbol}"][data-field="postMarketPrice"]')\
    .text
post_market_change = driver\
    .find_element(By.CSS_SELECTOR, f'[data-symbol="{ticker_symbol}"][data-field="postMarketChange"]')\
    .text
post_market_change_percent = driver\
    .find_element(By.CSS_SELECTOR, f'[data-symbol="{ticker_symbol}"][data-field="postMarketChangePercent"]')\
    .text\
    .replace('(', '').replace(')', '')

# initialize the dictionary
stock = {}

# stock price scraping logic omitted for brevity...

# add the scraped data to the dictionary
stock['regular_market_price'] = regular_market_price
stock['regular_market_change'] = regular_market_change
stock['regular_market_change_percent'] = regular_market_change_percent
stock['post_market_price'] = post_market_price
stock['post_market_change'] = post_market_change
stock['post_market_change_percent'] = post_market_change_percent

print(stock)

# close the browser and free up the resources
driver.quit()

{'regular_market_price': '197.12', 'regular_market_change': '-1.26', 'regular_market_change_percent': '-0.64%', 'post_market_price': '197.05', 'post_market_change': '-0.07', 'post_market_change_percent': '-0.04%'}
